# Aula prática: Destinação de Produtos à Fabricas
<sup>Exemplo do capítulo 8.3 do livro `Introdução à Pesquisa Operacional` de `Hillier e Lieberman`.</sup>

## Exercício 1

### Descrição do problema
A Cia. Produtos Melhores decidiu iniciar a produção de quatro produtos novos usando três fábricas que, no momento, têm excesso de capacidade produtiva. Os produtos requerem um esforço de produção comparável por unidade, de modo que a capacidade produtiva disponível das fábricas seja medida pelo número de unidades de qualquer produto que possa ser produzido diariamente, conforme dado na tabela 1. A tabela 2 fornece a taxa de produção diária necessária para atender às vendas projetadas. Cada fábrica é capaz de produzir qualquer um desses produtos, exceto a Fábrica 2, que não pode fabricar o produto 3. Os custos variáveis por unidade de cada produto diferem de fábrica para fábrica, conforme a tabela 3.

Tabela 1: Capacidade de produção disponível, por unidade do produto

| | Fábrica 1 | Fábrica 2 | Fábrica 3 |
|:---|:---:|:---:|:---:|
| Capacidade disponível | 75 | 75 | 45 |

Tabela 2: Demanda de produção diária

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Demanda | 20 | 30 | 30 | 40 |

Tabela 3: Custo unitário por produto

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Fábrica 1 | 41 | 27 | 28 | 24 |
| Fábrica 2 | 40 | 29 | - | 23 |
| Fábrica 3 | 37 | 30 | 27 | 21 |

A gerência precisa tomar uma decisão sobre como dividir a fabricação dos produtos entre as fábricas, e decidiu permitir a divisão da produção de um mesmo produto em mais de uma fábrica.

Escreva um modelo para determinar quais fábricas produzirão quais produtos e a que custo total.

### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [ ]:
custos = [(41, 27, 28, 24),
         (40, 29, 0, 23),
         (37, 30, 27, 21)
         ]

demanda = [20, 30, 30, 40]

capacidade = [75, 75, 45]

In [ ]:
model = Model(sense = MINIMIZE, solver_name=CBC)

x = [[model.add_var(var_type=INTEGER, name=f"x{i}_{j}", lb=0.0) for j in range(len(custos[0]))] for i in range(len(custos))]

model.objective = xsum(custos[i][j] * x[i][j] for i in range(len(custos)) for j in range(len(custos[0])))

for i in range((len(custos))):
  model += xsum(x[i][j] for j in range(len(custos[0]))) <= capacidade[i]

for j in range((len(custos[0]))):
  model += xsum(x[i][j] for i in range(len(custos))) == demanda[j]

model += x[1][2] == 0

save(model, "model.lp")

\Problem name: 

Minimize
OBJROW: 41 x0_0 + 27 x0_1 + 28 x0_2 + 24 x0_3 + 40 x1_0 + 29 x1_1 + 23 x1_3 + 37 x2_0 + 30 x2_1 + 27 x2_2
 + 21 x2_3
Subject To
constr(0):  x0_0 + x0_1 + x0_2 + x0_3 <= 75
constr(1):  x1_0 + x1_1 + x1_2 + x1_3 <= 75
constr(2):  x2_0 + x2_1 + x2_2 + x2_3 <= 45
constr(3):  x0_0 + x1_0 + x2_0 = 20
constr(4):  x0_1 + x1_1 + x2_1 = 30
constr(5):  x0_2 + x1_2 + x2_2 = 30
constr(6):  x0_3 + x1_3 + x2_3 = 40
constr(7):  x1_2 = 0
Bounds
Integers
x0_0 x0_1 x0_2 x0_3 x1_0 x1_1 x1_2 x1_3 x2_0 x2_1 
x2_2 x2_3 
End



In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3260.00

Solution:
x0_0 = 0.00
x0_1 = 30.00
x0_2 = 30.00
x0_3 = 0.00
x1_0 = 0.00
x1_1 = 0.00
x1_2 = 0.00
x1_3 = 15.00
x2_0 = 20.00
x2_1 = 0.00
x2_2 = 0.00
x2_3 = 25.00


## Exercício 2

Refaça o modelo anterior, dessa vez impedindo a divisão de produtos entre fábricas. Ou seja, cada produto deve ser integralmente produzido em uma única fábrica, para eliminar custos ocultos associados à divisão da produção. A cada fábrica deve ser destinado pelo menos um produto.

### Resolução

In [ ]:
model = Model(sense = MINIMIZE, solver_name=CBC)

x = [[model.add_var(var_type=BINARY, name=f"x{i}_{j}") for j in range(len(custos[0]))] for i in range(len(custos))]

model.objective = xsum((custos[i][j] * demanda[j])  * x[i][j] for i in range(len(custos)) for j in range(len(custos[0])))

for i in range((len(custos))):
  model += xsum(x[i][j] * demanda[j] for j in range(len(custos[0]))) <= capacidade[i]

for i in range((len(custos))):
  model += xsum(x[i][j] * demanda[j] for j in range(len(custos[0]))) >= 1

for j in range((len(custos[0]))):
  model += xsum(x[i][j] for i in range(len(custos))) == 1

model += x[1][2] == 0

save(model, "model2.lp")

\Problem name: 

Minimize
OBJROW: 820 x0_0 + 810 x0_1 + 840 x0_2 + 960 x0_3 + 800 x1_0 + 870 x1_1 + 920 x1_3 + 740 x2_0 + 900 x2_1 + 810 x2_2
 + 840 x2_3
Subject To
constr(0):  20 x0_0 + 30 x0_1 + 30 x0_2 + 40 x0_3 <= 75
constr(1):  20 x1_0 + 30 x1_1 + 30 x1_2 + 40 x1_3 <= 75
constr(2):  20 x2_0 + 30 x2_1 + 30 x2_2 + 40 x2_3 <= 45
constr(3):  20 x0_0 + 30 x0_1 + 30 x0_2 + 40 x0_3 >= 1
constr(4):  20 x1_0 + 30 x1_1 + 30 x1_2 + 40 x1_3 >= 1
constr(5):  20 x2_0 + 30 x2_1 + 30 x2_2 + 40 x2_3 >= 1
constr(6):  x0_0 + x1_0 + x2_0 = 1
constr(7):  x0_1 + x1_1 + x2_1 = 1
constr(8):  x0_2 + x1_2 + x2_2 = 1
constr(9):  x0_3 + x1_3 + x2_3 = 1
constr(10):  x1_2 = 0
Bounds
 0 <= x0_0 <= 1
 0 <= x0_1 <= 1
 0 <= x0_2 <= 1
 0 <= x0_3 <= 1
 0 <= x1_0 <= 1
 0 <= x1_1 <= 1
 0 <= x1_2 <= 1
 0 <= x1_3 <= 1
 0 <= x2_0 <= 1
 0 <= x2_1 <= 1
 0 <= x2_2 <= 1
 0 <= x2_3 <= 1
Integers
x0_0 x0_1 x0_2 x0_3 x1_0 x1_1 x1_2 x1_3 x2_0 x2_1 
x2_2 x2_3 
End



In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3290.00

Solution:
x0_0 = 0.00
x0_1 = 1.00
x0_2 = 1.00
x0_3 = 0.00
x1_0 = 1.00
x1_1 = 0.00
x1_2 = 0.00
x1_3 = 0.00
x2_0 = 0.00
x2_1 = 0.00
x2_2 = 0.00
x2_3 = 1.00
